In [1]:
import tensorflow as tf
import wandb
import modin.pandas as pd
import numpy as np
import pickle

In [2]:
import ray
ray.init()

2022-04-06 10:28:35,405	INFO services.py:1412 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '172.17.0.2',
 'raylet_ip_address': '172.17.0.2',
 'redis_address': None,
 'object_store_address': '/tmp/ray/session_2022-04-06_10-28-32_968631_32297/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-04-06_10-28-32_968631_32297/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2022-04-06_10-28-32_968631_32297',
 'metrics_export_port': 64952,
 'gcs_address': '172.17.0.2:60038',
 'address': '172.17.0.2:60038',
 'node_id': '38375c1b1e649b5cd6bd0af2f2d7f809dd89f0b256e7f48f1761edbe'}

In [3]:
# Read in data
def data():

    articles_df_original = pd.read_csv("data/articles.csv")
    customers_df_original = pd.read_csv("data/customers.csv")
    transactions_df_original = pd.read_csv("data/transactions_train.csv")

    return articles_df_original, customers_df_original, transactions_df_original

articles_df_original, customers_df_original, transactions_df_original = data()

In [4]:
# Create lookup dictionary
product_code_lookup = articles_df_original[["article_id","product_code"]]
product_code_lookup = product_code_lookup.set_index("article_id")["product_code"].to_dict()

# Create transactions matrix
transactions_matrix = transactions_df_original[["customer_id","article_id"]]

# Change article_id to product_code
transactions_matrix["article_id"] = transactions_matrix["article_id"].map(product_code_lookup)

# Rename article_id to product_code
transactions_matrix = transactions_matrix.rename(columns={"article_id": "product_code"})

# Sort by customer_id
transactions_matrix = transactions_matrix.sort_values(by=["customer_id"])

Please refer to https://modin.readthedocs.io/en/stable/supported_apis/defaulting_to_pandas.html for explanation.


In [5]:
# Split data into separate dataframes
n = 200000
list_df_transactions = [transactions_matrix[i:i+n] for i in range(0,transactions_matrix.shape[0],n)]

# Get quantity of items purchased by each user
list_df_transactions_quantity = []

for df in list_df_transactions:
    list_df_transactions_quantity.append(df.groupby(["customer_id","product_code"]).size().reset_index())

In [15]:
# Concatenate split dfs
df_transactions_quantity_original = pd.concat(list_df_transactions_quantity)
df_transactions_quantity_original = df_transactions_quantity_original.rename(columns={0:"quantity"})

In [16]:
with open('df_transactions_quantity','wb') as handle:
    pickle.dump(df_transactions_quantity_original, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [3]:
df_transactions_quantity_1 = pd.read_pickle("df_transactions_quantity")

Please refer to https://modin.readthedocs.io/en/stable/supported_apis/defaulting_to_pandas.html for explanation.


In [4]:
# Change dtype to object
df_transactions_quantity_1['product_code'] = df_transactions_quantity_1['product_code'].astype('object')

In [ ]:
df_transactions_quantity_1.head(10)

In [29]:
transactions_dict = {name: np.array(value) for name, value in df_transactions_quantity_1.items()}

In [35]:
transactions = tf.data.Dataset.from_tensor_slices(transactions_dict)

Error: Canceled future for execute_request message before replies were done